Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [3]:
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

http://vta2.amigocloud.com/api/otp/plan?mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.4002813%2C-121.9396906&toPlace=37.326763%2C-121.856533&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=15&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE

In [4]:
def query_string(start_lat, start_long, dest_lat, dest_long, bus = True, train = True, bike = True, walk_dist = 5000, walk_reluc = 2, visual = False):
    base = 'http://vta2.amigocloud.com'
    api = '/api/otp/plan?'
    date = 'date=2016-07-12'
    time = ''
    time = '&time=07%3A0'
    
    if bike:
        mode = '&mode=BICYCLE%2C'
    else:
        mode = '&mode=WALK%2C'

    if bus:
        mode = mode + 'BUSISH%2C'
    if train:
        mode = mode + 'TRAINISH'
    #mode = '&mode=WALK%2CBUSISH%2CTRAINISH'
    
    start = '&fromPlace=' + str(start_lat) + '%2C' + str(start_long)
    #start = '&fromPlace=37.389268%2C-121.879746'
    dest = '&toPlace=' + str(dest_lat) + '%2C' + str(dest_long)
    #dest = '&toPlace=37.4031595%2C-121.9865994'
    num_iter = '&numItineraries=3'
    
    walk = '&maxWalkDistance=' + str(walk_dist) + '&'
    
    bike = 'bikeSpeed=10'
    
    walk_reluctance = '&walkReluctance=' + str(walk_reluc)
    max_tran = '&maxTransfers=5'
    triangle = '&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE'
    arriveby = '&arriveBy=false'
    
    #Visualize doesn't work right at the moment
    if visual:
        api = '/planner?'
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluctance + max_tran + triangle + arriveby
    else:
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluctance + max_tran + triangle + arriveby

    return query

In [5]:
def print_transit(query_json):
    for a in query_json['plan']['itineraries']:
        print('start: '+ tclean(a['startTime']))
        print('end: '+ tclean(a['endTime']))
        print('duration: '+ str(a['duration']/60) + ' minutes')
        print('waitingTime: '+ str(a['waitingTime']/60) + ' minutes')
        print('transitTime: '+ str(a['transitTime']/60) + ' minutes')
        print('walkDistance: '+ str(a['walkDistance']/1000*.621371) + ' miles')
        print('transfers: '+ str(a['transfers']))
        for each in a['legs']:
            print('   StartTime: '+ tclean(each['startTime']) + " mode: " + each['mode'])
            print("   headsign: " + str(each.get('headsign')))
            print('   endTime:   ' + tclean(each['endTime']) + " duration: " + str(each['duration']/60))    

In [9]:
import os
os.getcwd()

'/Users/falconer_k/git/vta/OTP_Tester'

In [10]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')
known_starts=pd.read_csv('Known_Starts.csv')
known_dests=pd.read_csv('Known_Dests.csv')

In [11]:
df = pd.DataFrame();

#df['start_lat'] = rand_starts['Lat']
#df['start_long'] = rand_starts['Long']
#df['dest_lat'] = rand_dests['Lat']
#df['dest_long'] = rand_dests['Long']

df['start_lat'] = known_starts['Lat']
df['start_long'] = known_starts['Long']
df['start_loc']= known_starts['Desc']
df['dest_lat'] = known_dests['Lat']
df['dest_long'] = known_dests['Long']
df['dest_loc'] = known_dests['Desc']

In [15]:
def tclean(dto):
    return str(datetime.fromtimestamp(dto/1000))

In [12]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose


In [ ]:
#visuals = []
#visuals.append(query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row], walk_dist = 10000,visual = True))
#df['visual'] = visuals

In [13]:
def input_params(df, max_walk, walk_reluc_factor, bus_v = True, train_v = True, bike_v = True):
    row = 0
    times=[]
    results = []
    queries = []
    trips = []
    lowest = []
    
    for each in df.iterrows():
        query = query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], 
                             dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row],
                             walk_dist = max_walk, walk_reluc = walk_reluc_factor, bus = bus_v, train = train_v, bike = bike_v)
        queries.append(query)
        r = requests.get(query)
        query_json = json.loads(r.text)
        results.append(str(query_json))
        one_set_of_times= query_json['debugOutput']['pathTimes']
        times.append(one_set_of_times)
        row = row + 1
    df['times'] = times
    
    
    #Calculate Lowest trip length
    for each in df['times']:
        trips.append(len(each))
        if len(each) != 0:
            lowest.append(min(each))
        else:
            lowest.append(999)
    df['trips']=trips
    df['lowest']=lowest
    
    df['queries'] = queries
    df['results'] = results
    
    
    return df

In [18]:
series=input_params(df, max_walk = 5000, walk_reluc_factor = 2, train_v = True, bike_v = False, bus_v = False)
query = series['queries'][0]
r = requests.get(query)
query_json = json.loads(r.text)
print_transit(query_json)

start: 2016-07-12 07:00:00
end: 2016-07-12 08:47:09
duration: 107.15 minutes
waitingTime: 0.0 minutes
transitTime: 0.0 minutes
walkDistance: 5.2035289633287976 miles
transfers: 0
   StartTime: 2016-07-12 07:00:00 mode: WALK
   headsign: None
   endTime:   2016-07-12 08:47:09 duration: 107.15
start: 2016-07-12 07:02:34
end: 2016-07-12 09:11:28
duration: 128.9 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 2.5 minutes
walkDistance: 6.134764482733924 miles
transfers: 0
   StartTime: 2016-07-12 07:02:34 mode: WALK
   headsign: None
   endTime:   2016-07-12 08:07:29 duration: 64.91666666666667
   StartTime: 2016-07-12 08:07:30 mode: TRAM
   headsign: ALUM ROCK
   endTime:   2016-07-12 08:10:00 duration: 2.5
   StartTime: 2016-07-12 08:10:01 mode: WALK
   headsign: None
   endTime:   2016-07-12 09:11:28 duration: 61.45
start: 2016-07-12 07:17:34
end: 2016-07-12 09:26:28
duration: 128.9 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 2.5 minutes
walkDistance: 6.13

In [17]:
query

'http://vta2.amigocloud.com/api/otp/plan?date=2016-07-12&time=07%3A0&mode=WALK%2CTRAINISH&fromPlace=37.400281%2C-121.939691&toPlace=37.331997%2C-121.887286&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=2&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false'

In [ ]:
df.to_csv('series.csv')

In [ ]:
#'http://docs.python-guide.org/en/latest/scenarios/json/'
#query_json['plan']['itineraries'][x]['legs'][y]['legGeometry']

In [ ]:
#final = pd.DataFrame([each, every, sum(df['lowest']), sum(df['trips'])], columns = ['walk_dist','walk_reluc','lowest','trips'])
#finals.append(final)
#finals

In [ ]:
query

In [ ]:
series